In [1]:
from work.unet.data_functions import *

Using TensorFlow backend.


In [2]:
train_path = r'D:\Clarifruit\cherry_stem\data\unet_data\train'
test_path = r'D:\Clarifruit\cherry_stem\data\unet_data\test'
test_aug_path = os.path.join(test_path, 'aug')

target_size = (256, 256)
modes_dict = {'grayscale': 1, 'rgb': 3}
color_mode = 'grayscale'

x_folder_name = 'image'
y_folder_name = 'label'

x_prefix = 'image'
y_prefix = 'label'

weights_file_name = 'unet_cherry_stem.hdfs5'
input_size = (*target_size, modes_dict[color_mode])

data_gen_args = dict(rotation_range=0.2,
                     width_shift_range=0.05,
                     height_shift_range=0.05,
                     shear_range=0.05,
                     zoom_range=0.05,
                     horizontal_flip=True,
                     fill_mode='nearest')

In [3]:

train_gen = clarifruit_train_generator(batch_size=2,
                                       train_path=train_path,
                                       image_folder=x_folder_name,
                                       mask_folder=y_folder_name,
                                       aug_dict=data_gen_args,
                                       image_color_mode=color_mode,
                                       mask_color_mode=color_mode,
                                       image_save_prefix=x_prefix,
                                       mask_save_prefix=y_prefix,
                                       flag_multi_class=False,
                                       num_class=2,
                                       save_to_dir=None,
                                       target_size=target_size,
                                       seed=1)

test_gen = clarifruit_train_generator(batch_size=2,
                                      train_path=test_path,
                                      image_folder=x_folder_name,
                                      mask_folder=y_folder_name,
                                      aug_dict=data_gen_args,
                                      image_color_mode=color_mode,
                                      mask_color_mode=color_mode,
                                      image_save_prefix=x_prefix,
                                      mask_save_prefix=y_prefix,
                                      flag_multi_class=False,
                                      num_class=2,
                                      save_to_dir=None,
                                      target_size=target_size,
                                      seed=1)



In [4]:
model = unet(input_size=input_size)
model_checkpoint = ModelCheckpoint(weights_file_name, monitor='loss', verbose=1, save_best_only=True)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


D:\Clarifruit\cherry_stem\work\unet\model.py:51: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)


In [ ]:
model.fit_generator(train_gen, steps_per_epoch=100, epochs=3, callbacks=[model_checkpoint])

In [5]:
model.evaluate_generator(test_gen, steps=30, callbacks=[model_checkpoint], verbose=1)

Found 57 images belonging to 1 classes.
Found 57 images belonging to 1 classes.
30/30 [==============================] - 7s 248ms/step


[0.7101800825636265, 0.7009129928330243]

In [8]:
test = clarifruit_test_generator(batch_size=1,
                                 test_path=test_path,
                                 folder=x_folder_name,
                                 aug_dict=data_gen_args,
                                 save_prefix=x_prefix,
                                 color_mode=color_mode,
                                 save_to_dir=test_aug_path,
                                 target_size=target_size,
                                 seed=1)

for file in test.filenames:
    print(file)

AttributeError: 'generator' object has no attribute 'filenames'

In [9]:
def get_names(datagen):
    batches_per_epoch = datagen.samples // datagen.batch_size + (datagen.samples % datagen.batch_size > 0)
    for i in range(batches_per_epoch):
        batch = next(datagen)
        current_index = ((datagen.batch_index-1) * datagen.batch_size)
        if current_index < 0:
            if datagen.samples % datagen.batch_size > 0:
                current_index = max(0,datagen.samples - datagen.samples % datagen.batch_size)
            else:
                current_index = max(0,datagen.samples - datagen.batch_size)
        index_array = datagen.index_array[current_index:current_index + datagen.batch_size].tolist()
        img_paths = [datagen.filepaths[idx] for idx in index_array]
        yield img_paths
    
for name in get_names(test):
    print(name)

AttributeError: 'generator' object has no attribute 'samples'

In [6]:
test = clarifruit_test_generator(batch_size=1,
                                 test_path=test_path,
                                 folder=x_folder_name,
                                 aug_dict=data_gen_args,
                                 save_prefix=x_prefix,
                                 color_mode=color_mode,
                                 save_to_dir=test_aug_path,
                                 target_size=target_size,
                                 seed=1)

results = model.predict_generator(generator=test, steps=30, verbose=1)
saveResult(test_aug_path, results)

Found 57 images belonging to 1 classes.
30/30 [==============================] - 2s 52ms/step


d:\clarifruit\cherry_stem\cherry_venv\lib\site-packages\skimage\io\_io.py:141: UserWarning: D:\Clarifruit\cherry_stem\data\unet_data\test\aug\0_predict.png is a low contrast image
  warn('%s is a low contrast image' % fname)
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
d:\clarifruit\cherry_stem\cherry_venv\lib\site-packages\skimage\io\_io.py:141: UserWarning: D:\Clarifruit\cherry_stem\data\unet_data\test\aug\2_predict.png is a low contrast image
  warn('%s is a low contrast image' % fname)
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uin